In [ ]:

%load_ext autoreload
%autoreload 2

import pandas as pd
import itertools
import torch
from torch.utils.data import DataLoader

from modules.classifer_utils import ClassifierDataset, TrainingManager, GeneralNN

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")



In [ ]:
# Load the Titanic dataset.
titanic_train_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_test_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

print(f'titanic_train_csv_df shape: {titanic_train_csv_df.shape}')
print(f'titanic_test_csv_df shape: {titanic_test_csv_df.shape}')



In [ ]:

label_column_name = "survived"
[train_encoded_df, test_encoded_df] = ClassifierDataset.onehot_encode_datafames([titanic_train_csv_df, titanic_test_csv_df])


train_ds = ClassifierDataset(train_encoded_df, label_column_name)
test_ds = ClassifierDataset(test_encoded_df, label_column_name)

batch_size = int(len(train_ds) / 10)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, shuffle=True)
print(f'{len(train_ds)} training records in with batch size {batch_size}, {len(test_ds)} records for test')

print(f'train has {train_ds.get_feature_count()} features')
print(f'test has {test_ds.get_feature_count()} features')


dropoutRate = 0.2

first_training_record, _ = train_ds[0]
input_features = first_training_record.shape[-1]

model = GeneralNN( input_features, [32,16,8,1], dropoutRate )


print(model)


In [ ]:


training_mgr = TrainingManager(model)
training_mgr.train(train_dataloader, 100)



In [ ]:

training_mgr.eval(test_dataloader)
